In [1]:
#####################
# IMPORT STATEMENTS #
#####################
import pandas as pd
from collections import namedtuple
import numpy as np
from igraph import *
import json


In [2]:
# Read the dataframe
df = pd.read_csv('san_francisco-censustracts.csv')
# Select the data from December
df12 = df[df.month ==12]

In [3]:
# Load details about the location
with open('san_francisco_censustracts.json') as data_file:    
    location_data = json.load(data_file)

#Create display name and mean location coordinates mapping
display_name = {}
location_coordinates = {}
for data in location_data['features']:
    #print(id)
    id = int(data['properties']['MOVEMENT_ID'])
    display_name[id] = data['properties']['DISPLAY_NAME']
    location_coordinates[id] = np.mean(data['geometry']['coordinates'][0][0], 0)
#print(len(display_name))

In [4]:
# Get nodes and edges
sources = list(df12.sourceid.values)
destinations = list(df12.dstid.values)
sources_set = set(sources)
destinations_set = set(destinations)

In [5]:
nodes = sources_set.union(destinations_set)
node_count = len(nodes)
edges_count = df12.sourceid.count()
print(node_count)
print(edges_count)

1887
538998


In [6]:
#Create an edgelist
#edge_list = zip(sources,destinations)
edge_weights = list(df12.mean_travel_time)

In [17]:
edge_list = []
for i in range(len(sources)):
    sv = sorted([sources[i],destinations[i]])
    edge_list.append([sv[0],sv[1],edge_weights[i]])
    
    

In [18]:
len(edge_list)

538998

In [19]:
el_df = pd.DataFrame(np.matrix(edge_list))

In [24]:
el_df.columns = ['s','d','w']

In [25]:
el_df.head()

,s,d,w
0,1938.0,1952.0,596.69
1,629.0,1747.0,1454.81
2,1872.0,1919.0,1227.01
3,1871.0,1929.0,1005.54
4,81.0,2319.0,474.53


In [50]:
edge_list_non_dup = np.matrix(el_df.groupby(['s','d'],as_index=False).mean()).tolist()

In [51]:
# Write edgelist into file
with open('location_edge_list.txt','wb+') as f:
    for sdw in edge_list_non_dup:
        f.write('%d\t%d\t%f\n'%(sdw[0],sdw[1],sdw[2]))

In [52]:
# Write Display mapping into file
with open('display_name2.txt','wb+') as f:
    for k,v in display_name.items():
        f.write('%d\t%s\n'%(k,v.encode('ascii', 'ignore').decode('ascii')))

In [53]:
# Write mean location coordinates mapping into file
with open('mean_location_coordinates.txt','w+') as f:
    for k,v in location_coordinates.items():
        f.write('%d\t%f\t%f\n'%(k,v[0],v[1]))

In [54]:
len(display_name)

2710

In [55]:
max(edge_weights)

6439.18